In [90]:
import mysql.connector as sql
import glob
import pandas as pd
from datetime import datetime

In [91]:
df = pd.read_csv("/home/garage/workspace/price_update_crontab/upbit_krwbch_1min.csv")

In [92]:
df = df.tail(11)
print(df)

                timestamp      open     close      high       low    volume
1408  2020-09-18T09:43:00  271950.0  271950.0  271950.0  271950.0  1.339141
1409  2020-09-18T09:44:00  272000.0  272000.0  272000.0  272000.0  8.014833
1410  2020-09-18T09:45:00  272200.0  272400.0  272400.0  272200.0  8.077309
1411  2020-09-18T09:46:00  272400.0  272400.0  272400.0  272400.0  1.837297
1412  2020-09-18T09:47:00  272000.0  272000.0  272000.0  272000.0  4.262171
1413  2020-09-18T09:49:00  272500.0  272500.0  272500.0  272500.0  2.203790
1414  2020-09-18T09:50:00  272450.0  272450.0  272450.0  272450.0  7.340796
1415  2020-09-18T09:51:00  272600.0  272600.0  272600.0  272600.0  0.898549
1416  2020-09-18T09:52:00  272550.0  272500.0  272550.0  272500.0  7.973261
1417  2020-09-18T09:53:00  272250.0  272250.0  272250.0  272250.0  2.898549


In [93]:
db_connection = sql.connect(host='root.cqyptexqvznx.ap-northeast-2.rds.amazonaws.com',port=int(3306), database='garage_test', user='root', password='koscom!234')
db_cursor = db_connection.cursor()

In [94]:

query = 'SELECT max(base_time) FROM history_prc_upbit WHERE base_dt = %s and coin_type = %s'
db_cursor.execute(query,(datetime.today().strftime("%Y%m%d"),'bch',))
print(datetime.today().strftime("%Y%m%d"))
max_time = db_cursor.fetchone()[0]
print(max_time)


20200918
09:53:00


In [95]:
for i in range(len(df)):
    tmp = str(df.iloc[i]['timestamp']).replace("-","")
    tmp = tmp.split('T')
    if max_time >= tmp[1] :
        continue
    base_dt = tmp[0]
    coin_type = 'bch'
    base_time = tmp[1]
    print(base_time)
    open_price = df.iloc[i]['open']
    close_price = df.iloc[i]['close']
    high_price = df.iloc[i]['high']
    low_price = df.iloc[i]['low']
    volumn = df.iloc[i]['volume']

    query = "INSERT INTO history_prc_upbit VALUES('{}','{}','{}',{},{},{},{},{})".format(base_dt,base_time,coin_type,open_price,close_price,high_price,low_price,volumn)
    db_cursor.execute(query)
    print(query)
    db_connection.commit()

db_connection.close()